#Calculo de ángulos de torsón phi, psi y omega.

Se cargo la libreria de biopython para leer el archivo pdb.

Para leer las coordenadas atomicas se seleccionaron los residuos de interes. Para cada resiuos se extraen las coordenadas 3D de los atomoso clave:

N,CA, Y C del residuo seleccionado
-C del residuo anterior para $\phi$
N Y CA del residuo siguiente para $\psi$ y $\omega$

Se forma tres vectores
b1 = B - A, b2 = C - B, b3 = D - C
Se calculan las normales a los planos

n1= b1 X b2 ,  n2= b2 x b3


Se calcula el angulo para:

$\phi$, $\psi$ $\omega$


In [1]:
# libreria para leer el pdb
!pip install biopython

from google.colab import files
uploaded = files.upload()

import numpy as np
import pandas as pd
from Bio.PDB import PDBParser

#lectura del pdb
pdb_filename = list(uploaded.keys())[0]
parser = PDBParser(QUIET=True)
structure = parser.get_structure('prot', pdb_filename)
model = structure[0]

# Seleccionar la cadena C
chain_id = 'C'
chain = model[chain_id]

# Filtrar residuos ASN 460 a VAL 470 (se necesita uno antes y uno después para ángulos)
residues = [res for res in chain if 459 <= res.id[1] <= 471]

# funcion para calcular
def calc_dihedral_manual(p1, p2, p3, p4):
    b1 = p2 - p1
    b2 = p3 - p2
    b3 = p4 - p3
    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)
    b2_norm = b2 / np.linalg.norm(b2)
    m1 = np.cross(n1, b2_norm)
    x = np.dot(n1, n2)
    y = np.dot(m1, n2)
    return np.degrees(np.arctan2(y, x))

# calculos de psi ´phi y omega
result = []
for i in range(1, len(residues) - 1):
    try:
        res_prev = residues[i - 1]
        res = residues[i]
        res_next = residues[i + 1]

        # phi: C(i-1), N(i), CA(i), C(i)
        phi = calc_dihedral_manual(
            res_prev['C'].get_coord(),
            res['N'].get_coord(),
            res['CA'].get_coord(),
            res['C'].get_coord()
        )

        # psi: N(i), CA(i), C(i), N(i+1)
        psi = calc_dihedral_manual(
            res['N'].get_coord(),
            res['CA'].get_coord(),
            res['C'].get_coord(),
            res_next['N'].get_coord()
        )

        # omega: CA(i), C(i), N(i+1), CA(i+1)
        omega = calc_dihedral_manual(
            res['CA'].get_coord(),
            res['C'].get_coord(),
            res_next['N'].get_coord(),
            res_next['CA'].get_coord()
        )

        result.append({
            'Residue': f"{res.get_resname()} {res.id[1]}",
            'Phi': round(phi, 2),
            'Psi': round(psi, 2),
            'Omega': round(omega, 2)
        })
    except Exception as e:
        print(f"Residuo {res.get_resname()} {res.id[1]}: ERROR → {e}")
        continue

df = pd.DataFrame(result)
print(df.to_string(index=False))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.0 MB/s eta 0:00:00


Saving 3clq.pdb to 3clq.pdb
Residue       Phi       Psi       Omega
ASN 460 46.970001 25.379999  161.539993
VAL 461 83.849998 24.760000 -172.479996
PHE 462 79.360001 30.600000 -175.720001
ASN 463 64.540001 44.230000  175.750000
GLU 464 67.070000 41.500000 -177.690002
ALA 465 60.480000 36.130001 -175.690002
LEU 466 67.260002 51.860001 -169.110001
LYS 467 46.730000 40.639999  173.729996
ALA 468 72.949997 35.799999  175.330002
LEU 469 71.199997 49.900002  175.570007
VAL 470 78.739998 19.610001 -178.940002
